## MCP + LangGraph 튜토얼

reference: https://github.com/teddynote-lab/langgraph-mcp-agents/blob/master/MCP-HandsOn-KOR.ipynb


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### MultiServerMCPClient

사전에 `mcp_server_local.py`를 싱행해둡니다. 터미널을 열고 가상환경이 활성화 되어있는 상태에서 서버를 실행해 주세요.  
`async with`로 일시적인 Session 연결을 생성 후 해제

In [14]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_teddynote.messages import ainvoke_graph, astream_graph
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    answer = await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x113409620>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울!
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - 

In [15]:
answer

{'node': 'agent',
 'content': AIMessageChunk(content='는 항상 화창합니다.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-fd63d8c3-c9aa-44bb-9768-9aeaac8f801e', usage_metadata={'input_tokens': -50, 'output_tokens': 11, 'total_tokens': -39, 'input_token_details': {'cache_read': 0}}),
 'metadata': {'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:796c1d70-7fc6-f3dc-60a4-95257450cc34',
  'checkpoint_ns': 'agent:796c1d70-7fc6-f3dc-60a4-95257450cc34',
  'ls_provider': 'google_genai',
  'ls_model_name': 'models/gemini-1.5-pro',
  'ls_model_type': 'chat',
  'ls_temperature': 0.7}}

In [16]:
# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)

# 2. 명시적으로 연결 초기화(이 부분이 필요함)
await client.__aenter__()

# 이제 도구가 로드됨
print(client.get_tools())

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x1133f2b60>)]


Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)


In [17]:
agent = create_react_agent(model, client.get_tools())

await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울!
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울은 항상 맑습니다!

{'node': 'agent',
 'content': AIMessageChunk(content='은 항상 맑습니다!', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-3d3edd85-f486-4932-8445-d01dc22867ba', usage_metadata={'input_tokens': -50, 'output_tokens': 9, 'total_tokens': -41, 'input_token_details': {'cache_read': 0}}),
 'metadata': {'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:98ca71b7-e142-5cd8-429a-827b170b9760',
  'checkpoint_ns': 'agent:98ca71b7-e142-5cd8-429a-827b170b9760',
  'ls_provider': 'google_genai',
  'ls_model_name': 'models/gemini-1.5-pro',
  'ls_model_type': 'chat',
  'ls_temperature': 0.7}}

### Stdio 통신 방식

- Stdio 통신 방식은 로컬 환경에서 사용하기 위해 사용합니다.
- 통신을 위해 표준 입력/출력 사용

아래 python 경로는 수정

In [2]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langchain_teddynote.messages import astream_graph

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

server_params = StdioServerParameters(
    command="/usr/local/anaconda3/envs/mcp-agent/bin/python",
    args=["mcp_server_local.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # MCP 도구 로드
        tools = await load_mcp_tools(session)
        print(tools)
        
        # 에이전트 생성
        agent = create_react_agent(model, tools)
        
        # 에이전트 응답 스트리밍
        await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x115544180>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울!
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - 

## RAG를 구축한 MCP 서버 사용